In [1]:
import pandas as pd
from IPython.core.display import HTML

def display_dataframes(table_list):
    return HTML('<table><tr style="background-color:white;">' + 
               ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) + 
               '</tr></table>'
               )

## Dataframe of rows that changed between two other dataframes

In [2]:
df = pd.DataFrame({
    'ID': [
        'S1-12-106665', 
        'S1-12-106668', 
        'S1-12-106673', 
        'S1-12-106678', 
        'S1-12-106683', 
        'S1-12-106685',
        'S1-12-106686',
        'S1-12-106687',
        'S1-12-106688',
        ],
    'Type': [
        'P04',
        'P01',
        'P01',
        'P01',
        'P03',
        'P03',
        'P03',
        'P03',
        'P03',
        ],
    'Profile': [
        'HE200A',
        'HE200A',
        'HE200A',
        'HE200A',
        'HE200A',
        'HE200A',
        'HE200A',
        'HE200A',
        'HE200A',
        ],
    'Force[kN]': [
        8.2,
        16.7,
        16.0,
        12.5,
        7.6,
        3.9,
        3.1,
        3.1,
        6.9,

        ]
})
df1 = df.set_index('ID')

# Copy dataframe and change some values
df2 = df1.copy()
df2.at['S1-12-106668', 'Force[kN]'] = 10.4
df2.at['S1-12-106678', 'Force[kN]'] = 9.7
df2.at['S1-12-106687', 'Force[kN]'] = 8.1
df2.at['S1-12-106688', 'Force[kN]'] = 5.2



display_dataframes([df1, df2])

,Type,Profile,Force[kN]
ID,,,
S1-12-106665,P04,HE200A,8.2
S1-12-106668,P01,HE200A,16.7
S1-12-106673,P01,HE200A,16.0
S1-12-106678,P01,HE200A,12.5
S1-12-106683,P03,HE200A,7.6
S1-12-106685,P03,HE200A,3.9
S1-12-106686,P03,HE200A,3.1
S1-12-106687,P03,HE200A,3.1
S1-12-106688,P03,HE200A,6.9


In [3]:
# Create dataframe of rows where pile force 
# changed from df1 to df2 
df3 = df2[df1['Force[kN]'] != df2['Force[kN]']]
df3

,Type,Profile,Force[kN]
ID,,,
S1-12-106668,P01,HE200A,10.4
S1-12-106678,P01,HE200A,9.7
S1-12-106687,P03,HE200A,8.1
S1-12-106688,P03,HE200A,5.2


In [4]:
# Rename column with new pile forces 
df3 = df3.rename(columns={'Force[kN]': 'New_Force[kN]'})

# Insert column with old pile force
df3 = df3.merge(df1[['Force[kN]']], on='ID', how='inner')

# Rename column with old pile force
df3 = df3.rename(columns={'Force[kN]': 'Old_Force[kN]'})

# Rearrange columns so column with new pile force is rightmost
# df3 = df3[['Type', 'Profile', 'Old_Force[kN]', 'New_Force[kN]']]
    
# Add column with percentage change in pile force
df3['% change'] = 1.1 * (df3['New_Force[kN]'] - df3['Old_Force[kN]'])

df3

,Type,Profile,New_Force[kN],Old_Force[kN],% change
ID,,,,,
S1-12-106668,P01,HE200A,10.4,16.7,-6.93
S1-12-106678,P01,HE200A,9.7,12.5,-3.08
S1-12-106687,P03,HE200A,8.1,3.1,5.50
S1-12-106688,P03,HE200A,5.2,6.9,-1.87


In [5]:
def color_neg_pos(val):
    return f"color: {'red' if val < 0 else 'blue'}"

In [6]:
# Color changes
df3.style.applymap(color_neg_pos, subset=['% change'])

,Type,Profile,New_Force[kN],Old_Force[kN],% change
ID,,,,,
S1-12-106668,P01,HE200A,10.4,16.7,-6.93
S1-12-106678,P01,HE200A,9.7,12.5,-3.08
S1-12-106687,P03,HE200A,8.1,3.1,5.5
S1-12-106688,P03,HE200A,5.2,6.9,-1.87


## Customized merging of dataframes 

In [9]:
import pandas as pd

df1 = pd.DataFrame({'Invoice': ['20561', '20562', '20563', '20564'],
                    'Currency': ['EUR', 'EUR', 'EUR', 'USD']})
df2 = pd.DataFrame({'Ref': ['20561', 'INV20562', 'INV20563BG', '20564'],
                    'Amount': ['150', '175', '160', '180'],})

df3 = df2.copy()
df3['Invoice'] = [val for idx, val in enumerate(df1['Invoice']) if val in df2['Ref'][idx]]
df3 = df1.merge(df3[['Amount', 'Invoice']], on='Invoice')

display_dataframes([df1, df2, df3])

The first two dataframes are merged on Invoice/Ref column. The comparison checks if the Invoice number is contained in the Ref number. Other characters than the match are allowed to be in the string.